In [1]:
import numpy as np
import pandas as pd
import functools
import math
import fractions
import itertools as it
from numpy import linalg as LA

$\textbf{Технические вещи}$

In [2]:
#Поиск максимального модуля в массиве
def AbsMax(A):
    return np.max(np.array([np.max(A), -np.min(A)]))

In [3]:
#Спектральный радиус матрицы
def SpecRad (A):
        return AbsMax(np.linalg.eig(A)[0])

In [4]:
#Попытка замены вектора на вектор из целых
def integer_collinear_vector(numbers, max_denominator=1000000):
    t = np.array([])
    for i in range(len(numbers)):
        if numbers[i] != 0:
            a = np.array([numbers[i]])
            t = np.concatenate((t, a))
    norma = np.min(t)
    numbers = [num / norma for num in numbers]
    fracts = [fractions.Fraction(f).limit_denominator(max_denominator) for f in numbers]
    denoms = [f.denominator for f in fracts]
    denoms_gcd = functools.reduce(math.gcd, denoms)
    denoms_lcm = functools.reduce(lambda a, b: a*b, [d // denoms_gcd for d in denoms]) * denoms_gcd
    result_integers = [f.numerator * (denoms_lcm // f.denominator) for f in fracts]
    return result_integers

$\textbf{Итерационные методы}$

$\textbf{Метод Якоби}$

In [5]:
def yakob(A,X,F,N):
    Y = np.zeros(N)
    for i in range(N):
        for j in range(N):
            if j == i:
                continue
            Y[i] += -(X[j]*(A[i][j]))
        Y[i] += F[i]
        Y[i] = Y[i]/(A[i][i])
    X = Y
    return X

In [24]:
#Пример
#Размер матриц NxN
N = 3
A = np.array([[20.0,6.0,0.0],[6.0,6.0,-7.0],[0.0,-7.0,18.0]])
F = np.array([-6.0,-15.0,30.0])
X = np.array([0.0,0.0,1.0])
for i in range(10):
    X = yakob(A,X,F,N)
    print("Iteration:", i + 1)
    print("X: ", X)

Iteration: 1
X:  [-0.3        -1.33333333  1.66666667]
Iteration: 2
X:  [ 0.1        -0.25555556  1.14814815]
Iteration: 3
X:  [-0.22333333 -1.26049383  1.56728395]
Iteration: 4
X:  [ 0.07814815 -0.44816872  1.17647462]
Iteration: 5
X:  [-0.16554938 -1.20559442  1.49237883]
Iteration: 6
X:  [ 0.06167833 -0.59334198  1.19782439]
Iteration: 7
X:  [-0.12199741 -1.16421654  1.43592256]
Iteration: 8
X:  [ 0.04926496 -0.70275961  1.21391579]
Iteration: 9
X:  [-0.08917212 -1.13302987  1.39337126]
Iteration: 10
X:  [ 0.03990896 -0.78522807  1.22604394]


$\textbf{Метод Зейделя}$

In [7]:
def zeid(A,X,F,N):
    for i in range(N):
        X[i] = 0
        for j in range(N):
            if j == i:
                continue
            X[i] += -(X[j]*(A[i][j]))
        X[i] += F[i]
        X[i] = X[i]/(A[i][i])
    return X

In [26]:
#Пример
#Размер матриц NxN
N = 3
A = np.array([[18,0,6],[0,18,-7],[6,-7,6]])
F = np.array([-6,30,-15])
X = np.array([0.0,1.0,0.0])
for i in range(10):
    X = zeid(A,X,F,N)
    print("Iteration:", i + 1)
    print("X: ", X)

Iteration: 1
X:  [-0.33333333  1.66666667 -0.22222222]
Iteration: 2
X:  [-0.25925926  1.58024691 -0.39711934]
Iteration: 3
X:  [-0.20096022  1.51223137 -0.53476985]
Iteration: 4
X:  [-0.15507672  1.45870061 -0.6431059 ]
Iteration: 5
X:  [-0.1189647   1.41656993 -0.72837039]
Iteration: 6
X:  [-0.0905432   1.38341152 -0.79547669]
Iteration: 7
X:  [-0.06817444  1.35731462 -0.84829184]
Iteration: 8
X:  [-0.05056939  1.33677539 -0.88985932]
Iteration: 9
X:  [-0.03671356  1.32061026 -0.92257446]
Iteration: 10
X:  [-0.02580851  1.30788771 -0.9483225 ]


$\textbf{Метод верхней релаксации}$
<br>
$1<w<2$
<br>
$U_i^{s+1} =  U_i^s (1-w)  - \frac{w}{a_{ii}} (\sum_{j=1}^{i-1}a_{ij} u_j^{s+1} +\sum_{j=i+1}^{n}a_{ij} u_j^{s} - f_i)$

In [9]:
#Оптимальное значение для метода релаксации (если выполнен критерий)
def OptRel(A):
    D = np.diag(A)
    Q = np.identity(len(A))
    G = np.identity(len(A))
    for i in range(len(A)):
        Q[i][i] = 1/D[i]
        G[i][i] = D[i]
    R = -Q@(A - G)
    return (2/(1 + (1 - (SpecRad(R))**2)**0.5))
#Метод релаксации
def relax(A,X,F,N,w):
    for i in range(N):
        X[i] = X[i]*(1-w)*A[i][i]/w
        for j in range(N):
            if j == i:
                continue
            X[i] += -(X[j]*(A[i][j]))
        X[i] += F[i]
        X[i] = X[i]*w/(A[i][i])
    return X

In [25]:
#Размер матриц NxN
N = 3
A = np.array([[18,0,6],[0,18,-7],[6,-7,6]])
F = np.array([-6,30,-15])
X = np.array([0.0,1.0,0.0])
w = 1.368
for i in range(20):
    print("Iteration:", i + 1)
    X = relax(A,X,F,N,w)
    print("X: ", X)

Iteration: 1
X:  [-0.456    1.912    0.25536]
Iteration: 2
X:  [-0.40463616  1.71223552 -0.22770232]
Iteration: 3
X:  [-0.20326163  1.52875969 -0.61824316]
Iteration: 4
X:  [-0.09928084  1.38851107 -0.84060666]
Iteration: 5
X:  [-0.03614801  1.32182518 -0.95157327]
Iteration: 6
X:  [-0.00878012  1.28733135 -1.003229  ]
Iteration: 7
X:  [ 0.00470351  1.27254424 -1.02626552]
Iteration: 8
X:  [ 0.01024619  1.26573046 -1.03624526]
Iteration: 9
X:  [ 0.01275724  1.26292871 -1.04047942]
Iteration: 10
X:  [ 0.01376395  1.26170718 -1.042248  ]
Iteration: 11
X:  [ 0.01419995  1.26121582 -1.04297782]
Iteration: 12
X:  [ 0.0143723   1.26100838 -1.0432761 ]
Iteration: 13
X:  [ 0.0144449   1.26092603 -1.04339707]
Iteration: 14
X:  [ 0.01447334  1.26089198 -1.04344581]
Iteration: 15
X:  [ 0.0144851   1.26087858 -1.04346534]
Iteration: 16
X:  [ 0.01448968  1.26087312 -1.04347314]
Iteration: 17
X:  [ 0.01449155  1.26087098 -1.04347623]
Iteration: 18
X:  [ 0.01449227  1.26087012 -1.04347746]
Iteration:

In [14]:
#Орбита матрицы A при действии группы перестановок PAP^(-1)
def PermutationsOrbit(A):
    E = np.identity(len(A[0]))
    for P in it.permutations(E):
        print("Result: ")
        print(P@A@np.transpose(P))
        print("Permutation: ")
        print(P)

$\textbf{Метод простых итераций:}$

$u + \tau A u = \tau f + u$
<br>
$u^{s+1} = u^s  + \tau(f - A u^s)$
<br>
$r^s := f - A u^s$
<br>
$u^{s+1} = u^s  + \tau r^s$
<br>
$\delta^s = u^s - u^{*}$

In [14]:
#Шаг итерации, u - переменная,
# t - параметр МПИ, A - матрица, f - правая часть
def k(u,t,A,f):
    u = u+(f-A@u)*t
    return  u

$\textbf{Вычисление максимального по модулю собственного числа}$

$\lambda^{s+1} = \frac{(u^s,Au^s)}{(u^s,u^s)}$
<br>
$O(\frac{|\lambda_{beforemax}|}{|\lambda_{max}|})$

In [15]:
#Скалярное произведение двух векторов
def scal(x,y):
    m = x@(y.transpose())
    return m

In [21]:
u = np.array([1, 0])
t = 0.5
A = np.array([[5.0,-2.0],[-2,2]])
f = np.array([4, 5,6])
for j in range(5):
    print("Шаг №:", j+1)
    print("Вектор:",u)
    print("(u,Au):",scal(u,A@u))
    print("(u,u):",scal(u,u))
    print("lambda",scal(u,A@u)/scal(u,u))
    u = A@u

Шаг №: 1
Вектор: [1 0]
(u,Au): 5.0
(u,u): 1
lambda 5.0
Шаг №: 2
Вектор: [ 5. -2.]
(u,Au): 173.0
(u,u): 29.0
lambda 5.9655172413793105
Шаг №: 3
Вектор: [ 29. -14.]
(u,Au): 6221.0
(u,u): 1037.0
lambda 5.999035679845709
Шаг №: 4
Вектор: [173. -86.]
(u,Au): 223949.0
(u,u): 37325.0
lambda 5.999973208305425
Шаг №: 5
Вектор: [1037. -518.]
(u,Au): 8062157.0
(u,u): 1343693.0
lambda 5.9999992557823845


$\textbf{Метод вращений}$
$tg (2\alpha) = \frac{2a_{ij}}{a_{ii} - a{jj}}$

$\textbf{Вычисление невязки:}$

$F(u) = \sum_i(\sum_j a_{ij}u_j - f_i)^2$

In [30]:
def nevya(u,A,f,n):
    s = 0
    for i in range(n):
        k = 0
        for j in range(n):
            k += (A[i][j] * u[j] - f[i])
        s += k ** 2
    return s

In [31]:
n = 2
A = np.array([[1,1],[2,-1],[1,3],[3,1]])
f = np.array([3,0.2,7.0,5.0])
u = np.linalg.inv((A.transpose()@A))@A.transpose()@f
print("Вектор:",u)
print("Невязка:",nevya(u,A,f,n))

Вектор: [1.03741935 1.96774194]
Невязка: 9.054851196670137


$\textbf{Метод наискорейшего спуска}$

In [ ]:
#A - матрица, u - начальный ветор, f - правая часть, k - число итераций
def desc (A, u, f ,k):
    for i in range(k):
        r = A@u - f
        if(scal(r,r) == 0):
            return u
        t = scal(r, r)/scal(A@r, r)
        u = u - t*r
    return u

$\textbf{Метод минимальных невязок}$

In [2]:
#A - матрица, u - начальный ветор, f - правая часть, k - число итераций
def minnev (A, u, f ,k):
    for i in range(k):
        r = A@u - f
        if(scal(r,r) == 0):
            return u
        t = scal(A@r, r)/scal(A@r, A@r)
        u = u - t*r
    return u

$\textbf{Линейный МНК}$

Вычисление решения при матрице $A_{(m,n)}$, $rk(A) = n$:
<br>
$Au = f$
<br>
$A^{T} A x = A^{T} f$
<br>
$(A^{T}A)_{(n,n)}$
<br>
Тоесть это уже система с квадратной матрицей, которую мы решать умеем

In [20]:
#Выдаёт линейную систему, решение которой минимизирует сумму квадратов отклонений
def MNK_mat(A, f):
    return (np.transpose(A)@A, np.transpose(A)@f)
#Доверяем решение линейной системы python, вообще 
# можно использовать итерационные методы
def MNK_py(A, f):
    return np.linalg.solve(MNK_mat(A, f))

$\textbf{Интерполяция данных многочленом}$

In [ ]:
def SimplePolynomialInterpolation(x, y):
    A = np.array([[x[j]**i for i in range(len(y))] for j in range(len(x))])
    return np.linalg.solve(A, y)

Определение функции:

In [ ]:
def f:
    #Здесь надо написать функцию
def df:
    #Здесь надо написать её производную

$\textbf{Метод дихотомии:}$

In [1]:
def dych (a, b, f, k):
    x1 = a
    x2 = b
    for i in range(k):
        if(f(x1)*f((x1 + x2)/2) < 0):
            x2 = (x1 + x2)/2
        else:
            x1 = (x1 + x2)/2
    return x1

$\textbf{Метод простой итерации:}$

In [ ]:
def f:
    #Здесь надо написать функцию
def df:
    #Здесь надо написать её производную
    
    
def MPI(func, lambd, x, k, say = 0):
    for i in range(k):
        x = x - lambd(x)*func(x)
        if(say != 0):
            print (k, "th iteration, x is ", x)
    return x

